In [1]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "key_file_kim.json"

In [2]:
from google.cloud import language
language_client = language.LanguageServiceClient()

In [ ]:
# Run entity detection.
def detect_entities(txt, language_client):
    document = language.types.Document(
        content=txt,
        type=language.enums.Document.Type.PLAIN_TEXT,
        language="en")
    entities = language_client.analyze_entities(document).entities
    return entities

def article_entity_extract(qry,language_client):
    out=detect_entities(qry, language_client)

    res_ner=[{'name':out[idx].name,
      'type':out[idx].type,
      'salience':out[idx].salience,
      'mid':out[idx].metadata['mid'],
      'url':out[idx].metadata['wikipedia_url'],
      'mentions_unq':list(set([out[idx].mentions[i].text.content for i in range(len(out[idx].mentions))])),
      'mentions_len': len(out[idx].mentions)} 
      for idx in range(len(out))]
    return res_ner

def collection_entity_extract(art_collect, path, name, publisher, language_client):
    dt0=datetime.datetime.now()
    for idx in range(len(art_collect)):
        if art_collect[idx]['feed']['publisherName'] in publisher:
            txt_string=str(art_collect[idx]['Title']+ '\n'+str(art_collect[idx]['text']))
            
            assert(type(txt_string) is str)            
            art_collect[idx]['entities']=article_entity_extract(txt_string,language_client)
    with open(path+name+'_entities.txt', 'w') as f:
        json.dump({'key':art_collect},f,ensure_ascii=False)
    return art_collect

In [163]:
import json

with open('./RSS_IDs/rss_crawl2019-09-20 23-31-00.txt','r') as f:
    art_collect=json.load(f)
    #art_collect=json.loads(art1, object_hook=repl_keys)

In [95]:

publisher = ['BBC', 'Reuters']

In [101]:
len(art_collect['key'])

399

In [164]:
art_collect['key']=[rename_keys(art_collect['key'][i]) 
                    for i in range(len(art_collect['key'])) 
                    if art_collect['key'][i]['source'] in publisher]

In [161]:
def rename_keys(js1):
    js=js1.copy()
    js['site']=js['url']
    repl_key(js,'rss','feed')    
    js['url']=js['feed']['link']
    del js['feed']['link']
    js['feed']['publisherName']=js['source']
    del js['source']
    js['Title']=js['site']['Title']
    del js['site']['Title']
    js['text']=js['site']['Content']
    del js['site']['Content']
    
    dt_rss=js['feed']['published_parsed']
    dt_dt=datetime.datetime(dt_rss[0],dt_rss[1],dt_rss[2],dt_rss[3],dt_rss[4],dt_rss[5],dt_rss[6],)
    dt=int(datetime.datetime.timestamp(dt_dt)*1000)
    js['pub_date']={}
    js['pub_date']['$date']=dt
    return js

def repl_key(js, old, new):
    js[new]=js[old]
    del js[old]
    return js

In [137]:
art_collect['key'][1]['url']

'http://feeds.reuters.com/~r/reuters/businessNews/~3/1Nj45KUtLoY/wall-street-drops-after-china-cancels-trip-to-montana-farmland-idUSKBN1W51B6'

In [138]:
files=load_all_files('./RSS_IDs/')
files=list(set(files)-{'rss_id_list.txt','log.txt'})
len(files)

741

'20190924_0301'

In [123]:
def load_all_files(input_path):
    from os import walk

    f=[]
    for (_,_,filenames) in walk(input_path):
        f.extend(filenames)
        break
    filenames=list(set(filenames)-{'.DS_Store'})
    filenames=sorted(filenames,reverse=True)

    return filenames


In [136]:
files

['rss_crawl2019-09-22 06-01-31.txt',
 'rss_crawl2019-09-21 08-11-44.txt',
 'rss_crawl2019-09-24 03-01-31.txt',
 'rss_crawl2019-09-23 00-51-54.txt',
 'rss_crawl2019-09-25 04-41-50.txt',
 'rss_crawl2019-09-22 00-51-20.txt',
 'rss_crawl2019-09-23 09-31-50.txt',
 'rss_crawl2019-09-24 06-41-27.txt',
 'rss_crawl2019-09-23 10-01-35.txt',
 'rss_crawl2019-09-24 13-21-47.txt',
 'rss_crawl2019-09-25 02-31-13.txt',
 'rss_crawl2019-09-23 09-11-30.txt',
 'rss_crawl2019-09-24 13-11-34.txt',
 'rss_crawl2019-09-25 16-12-17.txt',
 'rss_crawl2019-09-23 08-01-24.txt',
 'rss_crawl2019-09-22 02-21-12.txt',
 'rss_crawl2019-09-23 07-51-17.txt',
 'rss_crawl2019-09-24 20-01-18.txt',
 'rss_crawl2019-09-21 17-01-14.txt',
 'rss_crawl2019-09-21 21-51-41.txt',
 'rss_crawl2019-09-24 04-11-35.txt',
 'rss_crawl2019-09-21 10-11-03.txt',
 'rss_crawl2019-09-24 08-01-43.txt',
 'rss_crawl2019-09-25 03-01-31.txt',
 'rss_crawl2019-09-25 09-51-13.txt',
 'rss_crawl2019-09-24 19-11-17.txt',
 'rss_crawl2019-09-23 17-11-50.txt',
 

In [239]:
time_st='20190923_0051'

payload=log_new_run(time_st,'ART',time_st,time_st,{}, )


In [260]:
with open('./RSS_IDs/'+files[4]) as f:
    art=json.load(f)
len(art['key'])

4

In [262]:
time_st

'20190923_0051'

In [269]:
art['key'][0]['source']

'Reuters'

In [274]:
len(art['key'][0]['url']['Content'])

2169

In [280]:

srs=['RT','BBC','BBC']

agg_srs(srs)

{'BBC': 2, 'RT': 1}

In [290]:
srs.count('BBC')

3

In [279]:

def agg_srs(srs):
    srs_sum={}
    for each in list(set(srs)):
        srs_sum[each]=srs.count(each)
    return srs_sum



In [ ]:
notes={'sources':agg_srs(srs),
       'avg_len':tot_len/len(art['key']),
       'nr_docs':len(art['key']),}

In [293]:
notes

{'avg_len': 4167.0, 'nr_docs': 4, 'sources': {'BBC': 3, 'Reuters': 1}}

In [265]:
'''for idx in range(len(art['key'])):
    es.delete('cnd-raw',id=time_st+'d'+str(idx))'''

In [306]:
payload
for file in files[:3]:
    
    st=file[9:25].replace(' ','_').replace('-','')
    start_dt=str(datetime.datetime.now())
    #start_dt=str(dt0)[:16].replace(' ','_').replace('-','').replace(':','')
    
    # INITIATE PARAMETERS FOR LOG
    srs=[]
    tot_len=0
    
    # READ FILE
    with open('./RSS_IDs/'+file) as f:
        art=json.load(f)
    
    if len(art['key'])>0:

        # CREATE ARTICLE RECORDS IN ES
        for idx in range(len(art['key'])):
            es.index('news-articles',id=st+'d'+str(idx),body=art['key'][idx])
            srs=srs+[art['key'][idx]['source']]
            tot_len=tot_len+len(art['key'][idx]['url'].get('Content',''))

        # CREATE NOTES
        notes={'sources':agg_srs(srs),
           'avg_len':int(tot_len/(len(art['key'])-srs.count('Investgate'))),
           'nr_docs':len(art['key']),}

        finish_dt=str(datetime.datetime.now())
        #finish_dt=str(dt0)[:16].replace(' ','_').replace('-','').replace(':','')    

        # LOG EVENTS
        payload=log_new_run(st,'ART',start_dt,finish_dt,notes, )
        es.index('news-log',id=st,body=payload)
    print(st,notes)

20190922_0601 {'sources': {'Reuters': 1}, 'avg_len': 3933, 'nr_docs': 1}
20190926_1603 {'sources': {'Investgate': 12, 'BBC': 2, 'Reuters': 7}, 'avg_len': 2838, 'nr_docs': 21}
20190921_0811 {'sources': {'BBC': 1, 'Reuters': 5}, 'avg_len': 2107, 'nr_docs': 6}


In [301]:
log_new_run(st,'ART',str(dt0),str(dt0),notes, )

{'ART_complete': '2019-09-27 08:18:20.767029',
 'ART_note': '{"sources": {"BBC": 1, "Reuters": 5}, "avg_len": 2107.8333333333335, "nr_docs": 6}',
 'ART_started': '2019-09-27 08:18:20.767029',
 'history': ['ART'],
 'run': '20190921_0811',
 'run_date': '20190927',
 'status': 'ART'}

In [284]:
es.indices.delete('cnd_raw')

{'acknowledged': True}

In [189]:
es_link='https://search-news-collection-zixpnvstipu7xweou5ixy4tpse.us-east-2.es.amazonaws.com/'

In [221]:
from elasticsearch import Elasticsearch, helpers

In [222]:
es=Elasticsearch(es_link)

In [313]:
es.search('news-log',{'query':{'match':{'run_date': '20190927'}}}, filter_path='hits.hits._source.run')

{'hits': {'hits': [{'_source': {'run': '20190921_0811'}},
   {'_source': {'run': '20190922_0601'}},
   {'_source': {'run': '20190926_1603'}}]}}

In [309]:
es.search('news-log',{'query':{'match':{'status': 'ART'}}})

TypeError: search() got an unexpected keyword argument 'filter'

In [304]:
def log_new_run(run_name,step,started,finished,stats):
    
    payload={'run':run_name,
             'status':step,
             'run_date':started.replace('-','')[:8],
             'history':[step],
             step+'_note':json.dumps(stats),
             step+'_started':started,
             step+'_complete':finished, 
             
            }
    return payload


def find_open_jobs(date,prev_step,max_jobs=3):
    # Search for any items in the collection with status prev_step
    
    # Return those as a list of jsons
    1
    
def update_to_running():
    1
def update_to_done():
    1
    
    


In [148]:
import datetime
dt0=datetime.datetime.now()
datetime.datetime.timestamp(dt0)*1000

1569533843050.334

In [266]:
dt0=datetime.datetime.now()
str(dt0)[:16].replace(' ','_').replace('-','').replace(':','')

'20190927_0747'

In [218]:
import requests

def create_doc(url, payload):
    session = requests.Session()
    session.headers={'Content=Type':'application/json'}
    query=json.dumps(payload)
    
    response=session.put(url,data=query)
    return response

def bulk_load(es_cnd,collection,doc_nm,payload_list):
    for idx in range(len(payload_list)):
        doc_nr=str(idx)
        doc_id=doc_name+'d'+doc_nr
        response=doc_load(es_cnd,collection,doc_id,payload_list[idx])
        
        res_out=json.loads(response.content)

        
def doc_load(es_cnd,collection,doc_id,payload):
    url=es_cnd+collection+'/_doc/'+doc_id
        
    #payload=json.load(payload)
    response=create_doc(url,payload)
    return response

#url=cnd+collection+'/_doc'+doc_id
#payload=json.loads(payload_list[idx])
#response=create_doc(url,payload)